## Imports

In [1]:
import sys
sys.path.append('python/')
import numpy as np
import lmfit as lf
import matplotlib.pyplot as plt
import scipy.optimize as opt
import dataPython as dp
from scipy.integrate import quad
import scipy.optimize as so
import scipy.special as ss

In [2]:
## Measured points
#Measured points from Noordermeer's paper:

In [3]:
#data = dp.getXYdata_wXYerr('data/NGC5533-rot-data_fmt.txt')
#r_dat = np.asarray(data['xx'])
#v_dat = np.asarray(data['yy'])
#v_err0 = np.asarray(data['ex'])
#v_err1 = np.asarray(data['ey'])

## Bulge

In [4]:
##Noordermeer's Bulge curve
data_bulge_nord = dp.getXYdata('data/NGC5533-bulge_fmt.txt')

#convert to numpy arrays
r_b_nord = np.asarray(data_bulge_nord['xx'])
v_b_nord = np.asarray(data_bulge_nord['yy'])

In [5]:
##Constants (Initial Guesses)
n = 2.7                                     #concentration parameter that describes the curvature of the profile in a radius-magnitude plot, n=4 is de Vaucoileurs profile
re = 2.6                                    #effective radius: radius which encompasses 50% of the light (kpc)
L = 3.27e10                                 #luminosity guess
G = 4.300e-6                                #gravitational constant (kpc/solar mass*(km/s)^2)
ups = 2.8                                   #mass-to-light ratio (from Rotation Curves of Sersic Bulges paper)
q = 0.33                                    #intrinsic axis ratio
i = 45*(np.pi/180)                          #inclination angle

##Definitions

x = np.linspace(0,100,100)
r = np.linspace(0,100,100)
m = np.linspace(0,100,100)

#Gamma Function
gamf = lambda x,n: ss.gammainc(2*n,x)*ss.gamma(2*n)-0.5*ss.gamma(2*n)
root = lambda x,n: so.brentq(gamf,0,500000,rtol=0.000001,args=(n,),maxiter=100) #come within 1% of exact root within 100 iterations

I0 = lambda n,re,L: L*(root(x,n)**(2*n))/(((re**2)*2*np.pi*n)*ss.gamma(2*n))
r0 = lambda n,re: re/root(x,n)**n
C = lambda n,re,L,G,ups,q,i: (4*G*q*ups*I0(n,re,L))/(r0(n,re)*np.float(n))*(np.sqrt((np.sin(i)**2)+(1/(q**2))*(np.cos(i)**2)))
e2 = lambda q: 1-(q**2)

#Inner Function
innerf = lambda x,m,n,re: ((np.exp(-np.power(x/r0(n,re), (1/n))))*(np.power(x/r0(n,re), ((1/n)-1))))/(np.sqrt((x**2)-(m**2)))
#Integrate Inner Function
g = lambda m,n,re: quad(innerf, m, np.inf,args=(m,n,re,))[0]

#Integrate outer function
h = lambda m,r,n,re,L,G,ups,q,i: C(n,re,L,G,ups,q,i)*g(m,n,re)*(m**2)/(np.sqrt((r**2)-((m**2)*(e2(q)))))

x = np.linspace(0.01, 10, 100)
y = np.zeros(np.shape(x))
hr = lambda m: h(m,r,n,re,L,G,ups,q,i)
for j,r in enumerate(x):
    y[j] = quad(h, 0, r, args=(r,n,re,L,G,ups,q,i,))[0]
    
##Main Function
def v(r,n,re,L,G,ups,q,i):
    return np.sqrt(y)

C:\Users\JB\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\integrate\quadpack.py:385: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  warnings.warn(msg, IntegrationWarning)
C:\Users\JB\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\integrate\quadpack.py:385: IntegrationWarning: The algorithm does not converge.  Roundoff error is detected
  in the extrapolation table.  It is assumed that the requested tolerance
  cannot be achieved, and that the returned result (if full_output = 1) is 
  the best which can be obtained.
  warnings.warn(msg, IntegrationWarning)


In [6]:
#SCIPY FITTING
#Setup
#Initial Guesses for n,re,L,G,ups,q,i. In this case, our initial guess is the true function.
p0 = [2.7,2.6,3.37e10,4.3e-6,2.8,0.33,45*(np.pi/180)] #Parameters should be in the same order here as they were when we defined our function.
bounds = [0,np.inf]                                   #KITTYADD bc sqrt can't be negative

#Do fit
s_fit = opt.curve_fit(v,r_b_nord,v_b_nord,p0,bounds=bounds,absolute_sigma=True) #absolute_sigma is set so that uncertainties aren't treated as percentages.
print(s_fit)                                          #If we uncomment this line, we see that s_fit is an array containing two arrays.

#Define parameters from fit. Our parameters are stored in the first array in our fit output, in the order they were listed in f.
s_n = s_fit[0][0]
s_re = s_fit[0][1]
s_L = s_fit[0][2]
s_G = s_fit[0][3]
s_ups = s_fit[0][4]
s_q = s_fit[0][5]
s_i = s_fit[0][6]

#Define error from fit.
s_cov = s_fit[1]                    #The second array in the fit output is a covariance matrix.
s_error = np.sqrt(np.diag(s_cov))   #The diagonals of the covariance matrix are the variances of individual parameters.
s_ne = s_error[0]                   #The errors will be in an order corresponding to the order of the parameters in their array
s_ree = s_error[1]
s_Le = s_error[2]
s_Ge = s_error[3]
s_upse = s_error[4]
s_qe = s_error[5]
s_ie = s_error[6]

##Main Function
def s_ve(r,s_ne,s_ree,s_Le,s_Ge,s_upse,s_qe,s_ie):
    return np.sqrt(y)

s_curve = s_ve 

#Print Values
print('n: '+str(s_n)+u' \u00B1 '+str(s_ne))
print('re: '+str(s_re)+u' \u00B1 '+str(s_ree))
print('L: '+str(s_L)+u' \u00B1 '+str(s_Le))
print('G: '+str(s_G)+u' \u00B1 '+str(s_Ge))
print('upsylon: '+str(s_ups)+u' \u00B1 '+str(s_upse))
print('q: '+str(s_q)+u' \u00B1 '+str(s_qe))
print('i: '+str(s_i)+u' \u00B1 '+str(s_ie))

ValueError: operands could not be broadcast together with shapes (100,) (846,) 

In [7]:
#LMFIT FITTING
#Setup
#weighdata = 1/sigdata                       #We will need weights for lmfit. This tells us how much to account for a single data point in the fit.
l_mod = lf.Model(v)                         #Tell lmfit that we want to model the function f
params = l_mod.make_params(n=2.7,ne=2.6,L=3.37e10,G=4.3e-6,ups=2.8,q=0.33,i=45*(np.pi/180))   #Give lmfit our initial guesses - again, the true function
params.add('n', value=2.7, min=0) #KITTYADD so lmfit doesn't guess negative values (bc G is so lose to zero)
params.add('ne', value=2.6, min=0)   #KITTYADD so lmfit doesn't guess negative values 
params.add('L', value=3.37e10, min=0) #KITTYADD so lmfit doesn't guess negative values (bc G is so lose to zero)
params.add('G', value=4.3e-6, min=0)   #KITTYADD so lmfit doesn't guess negative values 
params.add('ups', value=2.8, min=0) #KITTYADD so lmfit doesn't guess negative values (bc G is so lose to zero)
params.add('q', value=0.33, min=0)   #KITTYADD so lmfit doesn't guess negative values 
params.add('i', value=45*(np.pi/180), min=0) #KITTYADD so lmfit doesn't guess negative values (bc G is so lose to zero)

#Do fit
l_fit = l_mod.fit(v_b_nord, params, r=r_b_nord, nan_policy='omit') #Here is where the weights we set at the beginning come in.

#Define Stuff
l_dict = l_fit.best_values #l_fit has a lot of output. We want to use the final result.
l_n = l_dict['n']          #Dictionary items are called based on their name.
l_ne = l_dict['ne']          #So, we don't have to track the order of parameters.
l_L = l_dict['L']
l_G = l_dict['G']
l_ups = l_dict['ups']
l_q = l_dict['q']
l_i = l_dict['i']

#Create array to plot
l_curve = np.sqrt(l_G*l_M/r_bh_nord)  #KITTYCHANGE again, just equation only now with these second array things

l_fit #Display information about the fit

ValueError: operands could not be broadcast together with shapes (100,) (846,) 

In [ ]:
#Plotting

fig = plt.figure(figsize=(9.0,8.0))                #size of the plot

plt.errorbar(r_b_nord,v_b_nord,fmt='bo',label='Data') #Plot points (fmt='*o') with error bars
#plt.plot(r_b_nord,ycurve,label='Function')                      #Accepting default style gives us a solid line
plt.plot(r_b_nord,s_curve,linestyle='--',label='SciPy')         #Plot the scipy curve fit with a dashed line
plt.plot(r_b_nord,l_curve,linestyle='--',label='LmFit')

plt.legend()           #Tells our plot to show a legend
plt.show()             #Depending how your environment is set up, this line may not be necessary, but it won't break anything.